In [1]:
from joblib import Parallel, delayed

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import matplotlib.pyplot as plt
import numpy as np
from gensim.models.ldamulticore import LdaMulticore
import pandas as pd 
import pyLDAvis
import pyLDAvis.gensim

# Tokeniza cada texto en una lista de palabras
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations



In [2]:


#dataset_name = sys.argv[1]
#df = pd.read_csv(dataset_name)
df = pd.read_csv("datosProcesados.csv")


# Seleccionar las filas en las que la opinion sea positiva
X = df[df['__target__'] == 2]
X = X[X['airline'] == 'United']


# Paso todos los textos a una lista
textos = X.text.values.tolist()

# Paso cada texto de cada tweet de una string a una lista de palabras
data_words = list(sent_to_words(textos))

# Se crea el diccionario de las palabras; cada palabra unica contiene un identificador. Sirve para crear el corpus
id2word = corpora.Dictionary(data_words)

# Se crea el corpus
corpus = [id2word.doc2bow(text) for text in data_words]
#print(corpus[0])
# Cada palabra: (word_id, word_frequency). Si es (47,3) quiere decir que la palabra con id 47 aparece 3 veces en el documento




In [3]:
lda_model = LdaMulticore(corpus=corpus,  
    id2word=id2word,
    num_topics=5, 
    random_state=100,
    chunksize=100,
    passes=20,
    iterations=100,
    eval_every= None,
    per_word_topics=True,
    workers=8)
    
# La idea es tener la mejor similitud entre los textos de un topico mientras que se minimiza la similitud con los textos de otros
# La coherencia relaciona la distancia intracluster con la distancia intercluster
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()


# Imprime los topicos; por cada topico muestra su id y luego las palabras mas frecuentes con la frecuencia de esa palabra en ese topico
print(lda_model.print_topics())

[(0, '0.092*"unit" + 0.033*"flight" + 0.013*"great" + 0.012*"thank" + 0.011*"servic" + 0.009*"custom" + 0.009*"way" + 0.008*"best" + 0.008*"good" + 0.008*"ever"'), (1, '0.130*"unit" + 0.085*"thank" + 0.014*"time" + 0.014*"flight" + 0.010*"help" + 0.010*"great" + 0.010*"sent" + 0.008*"dm" + 0.007*"respons" + 0.006*"work"'), (2, '0.099*"unit" + 0.052*"thank" + 0.018*"flight" + 0.017*"get" + 0.015*"help" + 0.011*"much" + 0.010*"home" + 0.009*"got" + 0.009*"appreci" + 0.008*"follow"'), (3, '0.065*"unit" + 0.040*"thank" + 0.017*"flight" + 0.012*"get" + 0.010*"good" + 0.010*"bag" + 0.010*"help" + 0.010*"amp" + 0.008*"time" + 0.007*"custom"'), (4, '0.063*"unit" + 0.035*"thank" + 0.010*"amaz" + 0.008*"tri" + 0.008*"servic" + 0.007*"thx" + 0.007*"make" + 0.007*"sure" + 0.007*"ua" + 0.007*"help"')]


In [4]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.039967  0.117954       1        1  28.202554
2     -0.013376 -0.117067       2        1  28.010527
1     -0.110675 -0.013899       3        1  21.127053
3      0.068395 -0.042071       4        1  12.558398
4      0.095623  0.055084       5        1  10.101468, topic_info=         Term        Freq       Total Category  logprob  loglift
0       thank  187.000000  187.000000  Default  30.0000  30.0000
199       get   27.000000   27.000000  Default  29.0000  29.0000
1        unit  402.000000  402.000000  Default  28.0000  28.0000
133      time   17.000000   17.000000  Default  27.0000  27.0000
92       sent   10.000000   10.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
143      much    1.480917   18.848916   Topic5  -5.6596  -0.2513
407      crew    1.479650    9.320989   Topic5  -5.6604   0.4520
504      hour    1.479204    6.917667   Topic5  -5.6607   0.7499
21       know    1.479091    9.722830   Topic5  -5.6608   0.4094
224  daughter    1.479025    6.733065   Topic5  -5.6609   0.7768

[342 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1039      5  0.703628      aaron
44        2  0.652534        abl
44        4  0.261014        abl
44        5  0.130507        abl
1242      4  0.685374  accompani
...     ...       ...        ...
239       1  0.400421         ye
239       2  0.266947         ye
239       4  0.133474         ye
239       5  0.266947         ye
1159      4  0.685374  yesterday

[530 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 4, 5])